In [22]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import mean_squared_error
import math

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GRU
import time

tf.random.set_seed(42)

In [23]:
h_a = np.loadtxt('../../Data/data_39_highSI.txt', dtype=float)

h_a = h_a[h_a>0]
h_a = h_a[1::20]

avr = np.average(h_a)
h_a = h_a/avr
h_a = h_a.reshape(-1,1)

In [24]:
N_train = 7500       # train len for each iteration
N_test = 2500      # test len
k = (len(h_a) - N_train) // N_test
N_total = N_train+k*N_test       # total sample len
out_size = 1
timesteps = 21

Y_test = np.zeros((N_total-N_train - k*(timesteps-1), out_size), dtype=float)
Y_pred = np.zeros((N_total-N_train - k*(timesteps-1), out_size), dtype=float)

In [25]:
model = Sequential([
    GRU(50, return_sequences=True, input_shape=(timesteps-out_size,1)),
    Dropout(0.1),

    # GRU(20, return_sequences=True),

    # GRU(20, return_sequences=True),

    GRU(50, return_sequences=False),
    Dropout(0.1),

    Dense(out_size),

])
model.compile(optimizer='adam', loss='mean_absolute_error')

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_6 (GRU)                 (None, 20, 50)            7950      
                                                                 
 dropout (Dropout)           (None, 20, 50)            0         
                                                                 
 gru_7 (GRU)                 (None, 50)                15300     
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 23,301
Trainable params: 23,301
Non-trainable params: 0
_________________________________________________________________


In [32]:
for idx in range(1,5):
    print('idx = ' + str(idx))

    train_data = h_a[idx*N_test : N_train+idx*N_test].reshape(-1,1)
    test_data = h_a[N_train + idx*N_test : (idx+1)*N_test + N_train].reshape(-1,1)

    train_data_timesteps=np.array([[j for j in train_data[i:i+timesteps]] for i in range(0,len(train_data)-timesteps+1)])[:,:,0]
    test_data_timesteps=np.array([[j for j in test_data[i:i+timesteps]] for i in range(0,len(test_data)-timesteps+1)])[:,:,0]

    x_train, y_train = train_data_timesteps[:,:timesteps-out_size],train_data_timesteps[:,timesteps-out_size:]
    x_test, y_test = test_data_timesteps[:,:timesteps-out_size],test_data_timesteps[:,timesteps-out_size:]

    print('Start training ............')
    st = time.time()
    early_stop = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 10)
    history = model.fit(
        x_train,
        y_train,
        epochs = 1,
        batch_size = 128,
        verbose = 1,
    )
    et = time.time()
    print('Execution time = ' + str(et - st) + ' sec')

    # y_train_pred = model.predict(x_train).reshape(-1,1)
    print('Start predicting ............')
    st = time.time()
    y_test_pred = model.predict(x_test)
    et = time.time()
    print('Execution time = ' + str(et - st) + ' sec')
    
    Y_test[i*(N_test-timesteps+1):(i+1)*(N_test-timesteps+1), :] = y_test
    Y_pred[i*(N_test-timesteps+1):(i+1)*(N_test-timesteps+1), :] = y_test_pred
    print('Done! \n')

i = 1
Start training ............
59/59 [==============================] - 1s 14ms/step - loss: 0.0402
Execution time = 0.9034099578857422 sec
Start predicting ............
78/78 [==============================] - 0s 3ms/step
Execution time = 0.2674858570098877 sec
Done! 

i = 2
Start training ............
59/59 [==============================] - 1s 14ms/step - loss: 0.0379
Execution time = 0.8986551761627197 sec
Start predicting ............
78/78 [==============================] - 0s 3ms/step
Execution time = 0.2597157955169678 sec
Done! 

i = 3
Start training ............
59/59 [==============================] - 1s 15ms/step - loss: 0.0378
Execution time = 0.920050859451294 sec
Start predicting ............
78/78 [==============================] - 0s 3ms/step
Execution time = 0.2543056011199951 sec
Done! 

i = 4
Start training ............
59/59 [==============================] - 1s 14ms/step - loss: 0.0369
Execution time = 0.8945791721343994 sec
Start predicting ............
78/78 

In [28]:
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
import math

MSE = mean_squared_error(Y_test, Y_pred)
RMSE = math.sqrt(MSE)
MAE = mean_absolute_error(Y_test, Y_pred)
MAPE = mean_absolute_percentage_error(Y_test, Y_pred)

print('MSE = ', MSE)
print('RMSE = ', RMSE)
print('MAE = ', MAE)
print('MAPE = ', MAPE)

MSE =  1.0231216942377378e-05
RMSE =  0.003198627352846433
MAE =  0.00032053857955541065
MAPE =  0.00036043715812944906


In [29]:
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
import math

MSE = [0 for j in range(out_size)]
RMSE = [0 for j in range(out_size)]
MAE = [0 for j in range(out_size)]
MAPE = [0 for j in range(out_size)]

for i in range(out_size):
    MSE[i] = mean_squared_error(Y_test[:,i], Y_pred[:,i])
    RMSE[i] = math.sqrt(MSE[i])
    MAE[i] = mean_absolute_error(Y_test[:,i], Y_pred[:,i])
    MAPE[i] = mean_absolute_percentage_error(Y_test[:,i], Y_pred[:,i])

print('MSE = ', MSE)
print('RMSE = ', RMSE)
print('MAE = ', MAE)
print('MAPE = ', MAPE)

MSE =  [1.0231216942377378e-05]
RMSE =  [0.003198627352846433]
MAE =  [0.00032053857955541065]
MAPE =  [0.00036043715812944906]


In [30]:
# np.save("../Results/Multi_step/highSI/GRU_test", Y_test)
# np.save("../Results/Multi_step/highSI/GRU_pred", Y_pred)

In [31]:
# MAE = np.zeros((out_size))

# for j in range(out_size):
#     Y_t = Y_test[:,j]
#     Y_p = Y_pred[:,j]

#     MAE[j] = mean_absolute_error(Y_t, Y_p)

#     np.savetxt("../../Data/Multi-step-GRU/highSI_39/real_" + str(j)+ ".txt", Y_t, delimiter = '\n')
#     np.savetxt("../../Data/Multi-step-GRU/highSI_39/predicted_" + str(j)+ ".txt", Y_p, delimiter = '\n')

# print(MAE)
# np.savetxt("../../Data/Multi-step-GRU/highSI_39/MAE.txt", MAE, delimiter='\n')